<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-25 13:00:33
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.38 C
-------------------
Today PnL: 41.80 K (0.3%)
Current PnL: -17.81 L (-12.12%)
CY Booked + Current PnL: -9.08 L (-6.18%)
-------------------
Total profit:  3.69 L
Total loss:  -21.51 L
-------------------
Total Booked + Current PnL: 17.90 L (14.91%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.35%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 76.93 L (55.94%)
Deployed:  1.20 C
Current:  1.38 C
CAGR/XIRR %: 8.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-17.42,H-LC,7.19,114456.0,10144.0,6501.0,1.58,9.72,5.68,15.96,9.0,1.56,0.85,28.27,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,14.41,H-MC,14.74,163305.0,37105.0,9063.0,1.67,29.40,5.55,36.59,108.0,4.09,1.22,45.65,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-13.42,H-LC,7.88,172964.0,16787.0,9305.0,-0.76,10.75,5.38,16.71,22.0,1.80,1.29,30.06,X40N,BTT,HEALTHCARE
37,HINDUNILVR,2413.81,2723.00,-6.09,H-LC,7.75,295221.0,24874.0,9742.0,0.23,9.20,3.30,12.81,24.0,2.55,2.20,23.47,XY25,NTT,FMCG
68,SBIN,760.30,863.00,-12.30,M-LC,3.97,216995.0,15516.0,11696.0,0.32,7.70,5.39,13.51,60.0,1.33,1.62,21.27,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,NATIONALUM,189.63,247.44,-49.46,H-MC,1.97,100997.0,493.0,30148.0,0.89,0.49,29.85,30.49,79.0,0.02,0.75,33.05,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-39.61,H-LC,3.08,144981.0,-4951.0,19543.0,-1.79,-3.30,13.48,9.74,4.0,-0.25,1.08,2.73,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-3.47,H-LC,3.38,191412.0,-8898.0,68870.0,1.77,-4.44,35.98,29.94,16.0,-0.13,1.43,32.24,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-12.55,H-LC,3.57,269217.0,3791.0,53009.0,-1.07,1.43,19.69,21.40,11.0,0.07,2.01,8.62,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-17.72,H-LC,4.19,219474.0,14974.0,26447.0,0.58,7.32,12.05,20.25,10.0,0.57,1.64,16.69,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.45,H-LC,6.05,168905.0,-32481.0,120091.0,-0.82,-16.13,71.10,43.51,27.0,-0.27,1.26,17.57,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,INFY,1461.46,2275.00,-16.01,H-LC,11.97,275051.0,13450.0,132162.0,3.30,5.14,48.05,55.67,3.0,0.10,2.05,11.56,X40,BTT,IT
80,TCS,3794.03,4998.00,-25.80,H-LC,16.10,245528.0,-50406.0,144321.0,3.07,-17.03,58.78,31.73,1.0,-0.35,1.83,4.82,X40,BTT,IT
34,HCLTECH,1562.67,1943.91,3.66,H-LC,10.55,183854.0,-6792.0,53299.0,2.78,-3.56,28.99,24.40,8.0,-0.13,1.37,11.81,X40,ATH,IT
92,WIPRO,243.46,420.00,-10.16,M-LC,7.78,158007.0,7062.0,102389.0,2.50,4.68,64.80,72.51,53.0,0.07,1.18,10.98,XR,NTT,IT
38,HINDZINC,471.44,730.22,16.37,M-LC,4.01,189421.0,-15655.0,128219.0,2.48,-7.63,67.69,54.89,52.0,-0.12,1.41,14.62,X5K,ATH,METALS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.0,-43.66,L-SC,25.80,83443.0,-30106.0,70159.0,-1.96,-26.51,84.08,35.27,268.0,-0.43,0.62,104.23,XR,NTT,HOTELS
60,RAJESHEXPO,517.65,518.0,1672.73,L-SC,7.96,48378.0,-88799.0,88890.0,-1.82,-64.73,183.74,0.07,267.0,-1.00,0.36,20.32,OX40N,NTT,JEWELLERY
75,SURYODAY,189.78,240.0,42.54,H-SC,5.41,101752.0,-48174.0,87853.0,-1.81,-32.13,86.34,26.46,135.0,-0.55,0.76,30.35,XR,NTT,BANKS
47,ITC,411.90,452.0,-39.61,H-LC,3.08,144981.0,-4951.0,19543.0,-1.79,-3.30,13.48,9.74,4.0,-0.25,1.08,2.73,X40,NTT,FMCG
6,ASIANTILES,75.41,137.0,6816.67,L-SC,6.81,75884.0,-17926.0,94544.0,-1.69,-19.11,124.59,81.67,269.0,-0.19,0.57,46.95,XR,NTT,CERAMICS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,99.79,M-SC,8.85,86106.0,-14671.0,14767.0,0.42,-14.56,17.15,0.09,245.0,-0.99,0.64,13.12,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,89.03,H-SC,15.74,119043.0,-23436.0,42963.0,0.87,-16.45,36.09,13.70,163.0,-0.55,0.89,40.50,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-18.38,H-SC,3.09,116343.0,-27460.0,54204.0,-1.00,-19.10,46.59,18.60,149.0,-0.51,0.87,28.39,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,112.05,M-SC,1.85,138979.0,-35580.0,35606.0,-1.12,-20.38,25.62,0.02,221.0,-1.00,1.04,20.89,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.44,H-SC,3.53,210915.0,-58752.0,95080.0,0.74,-21.79,45.08,13.47,138.0,-0.62,1.57,6.68,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.80,H-LC,16.10,245528.0,-50406.0,144321.0,3.07,-17.03,58.78,31.73,1.0,-0.35,1.83,4.82,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.01,H-LC,11.97,275051.0,13450.0,132162.0,3.30,5.14,48.05,55.67,3.0,0.10,2.05,11.56,X40,BTT,IT
47,ITC,411.90,452.00,-39.61,H-LC,3.08,144981.0,-4951.0,19543.0,-1.79,-3.30,13.48,9.74,4.0,-0.25,1.08,2.73,X40,NTT,FMCG
89,VBL,492.64,671.64,-8.30,H-LC,8.95,278684.0,13644.0,82658.0,0.27,5.15,29.66,36.33,5.0,0.17,2.08,19.13,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.42,H-LC,12.56,198939.0,-17580.0,110491.0,0.75,-8.12,55.54,42.91,7.0,-0.16,1.48,4.18,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,14.41,H-MC,14.74,163305.0,37105.0,9063.0,1.67,29.40,5.55,36.59,108.0,4.09,1.22,45.65,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.35,H-LC,10.29,231422.0,46941.0,23420.0,0.51,25.44,10.12,38.14,38.0,2.00,1.73,38.75,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,14.41,H-MC,14.74,163305.0,37105.0,9063.0,1.67,29.40,5.55,36.59,108.0,4.09,1.22,45.65,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,5.01,191880.0,27210.0,40755.0,-0.06,16.52,21.24,41.28,89.0,0.67,1.43,25.83,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,8.64,H-SC,10.90,129364.0,4067.0,54941.0,-1.06,3.25,42.47,47.09,141.0,0.07,0.96,61.91,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-2.34,H-MC,6.95,205035.0,13293.0,82732.0,1.01,6.93,40.35,50.08,99.0,0.16,1.53,14.20,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,13.13,177100.0,26187.0,31116.0,1.24,17.35,17.57,37.97,93.0,0.84,1.32,47.05,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,13.13,177100.0,26187.0,31116.0,1.24,17.35,17.57,37.97,93.0,0.84,1.32,47.05,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.79,H-SC,12.30,140174.0,16838.0,116919.0,-0.96,13.65,83.41,108.45,119.0,0.14,1.05,39.06,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.18,M-SC,5.52,97672.0,6174.0,72580.0,0.10,6.75,74.31,86.07,223.0,0.09,0.73,41.29,XR,NTT,DURABLES
51,KPIGREEN,497.21,731.35,8.64,H-SC,10.90,129364.0,4067.0,54941.0,-1.06,3.25,42.47,47.09,141.0,0.07,0.96,61.91,MH,ATH,POWER
92,WIPRO,243.46,420.00,-10.16,M-LC,7.78,158007.0,7062.0,102389.0,2.50,4.68,64.80,72.51,53.0,0.07,1.18,10.98,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,75.41,137.00,6816.67,L-SC,6.81,75884.0,-17926.0,94544.0,-1.69,-19.11,124.59,81.67,269.0,-0.19,0.57,46.95,XR,NTT,CERAMICS
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.80,83443.0,-30106.0,70159.0,-1.96,-26.51,84.08,35.27,268.0,-0.43,0.62,104.23,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-0.47,H-SC,16.08,92631.0,-8332.0,102867.0,0.16,-8.25,111.05,93.63,148.0,-0.08,0.69,37.97,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-15.54,H-SC,12.71,95760.0,-2220.0,24093.0,1.69,-2.27,25.16,22.32,152.0,-0.09,0.71,38.89,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-10.40,H-LC,6.69,96260.0,-1485.0,38879.0,-0.07,-1.52,40.39,38.26,35.0,-0.04,0.72,26.05,X40,BTT,FINANCE


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,25.80,83443.0,-30106.0,70159.0,-1.96,-26.51,84.08,35.27,268.0,-0.43,0.62,104.23,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,13.13,177100.0,26187.0,31116.0,1.24,17.35,17.57,37.97,93.0,0.84,1.32,47.05,AR,ATH,AUTO
28,DMART,3764.92,5201.00,-7.35,H-LC,10.29,231422.0,46941.0,23420.0,0.51,25.44,10.12,38.14,38.0,2.00,1.73,38.75,X40N,NTT,FMCG
15,BLUESTARCO,1646.70,2326.38,11.16,H-MC,5.01,191880.0,27210.0,40755.0,-0.06,16.52,21.24,41.28,89.0,0.67,1.43,25.83,X40N,ATH,AC
37,HINDUNILVR,2413.81,2723.00,-6.09,H-LC,7.75,295221.0,24874.0,9742.0,0.23,9.20,3.30,12.81,24.0,2.55,2.20,23.47,XY25,NTT,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.89
1,25,41.50
2,50,71.13


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.37
LC    36.34
MC    26.31
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.37
X40      14.41
X40N     14.01
XY25     11.78
XR        9.06
AR        8.45
OX40N     7.42
X5K       2.26
MH        1.71
X200      1.43
SR        1.12
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.09
H-MC    23.41
H-SC    22.64
M-SC    13.18
M-LC     7.20
M-MC     2.57
L-SC     1.55
L-LC     1.05
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.49,-0.02,30.65
IT,12.10,-15.03,72.48
FINANCE,8.04,-20.74,65.41
BANKS,7.55,-12.68,60.53
MISC,5.90,-16.50,83.28
PAINTS,5.48,-18.85,42.27
ELECTRICAL,4.76,-10.23,57.96
AUTO,4.54,-11.02,59.32
AC,3.69,2.21,30.91


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2831226.0,23
XR,1078323.0,13
AR,1074878.0,10
X40,804439.0,11
X40N,490536.0,11
XY25,466851.0,8
OX40N,464729.0,10
SR,193081.0,2
X5K,134720.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2736644.0,27
M-SC,1479549.0,17
H-MC,1238925.0,18
H-LC,1135906.0,19
M-LC,413033.0,5
M-MC,333132.0,2
L-SC,253593.0,3
L-MC,58500.0,1
L-LC,43460.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      929465.0      6
M-SC       XY24      868502.0      7
H-SC       AR        654574.0      5
           XR        516933.0      6
H-LC       X40       508295.0      6
H-MC       XY24      482492.0      5
M-MC       XY24      333132.0      2
H-LC       AR        229918.0      2
H-MC       X40       213178.0      4
H-SC       OX40N     199716.0      4
H-MC       XY25      198584.0      2
H-SC       SR        193081.0      2
           X40N      187934.0      3
H-LC       X40N      184854.0      5
M-LC       XY24      170729.0      2
L-SC       XR        164703.0      2
M-SC       AR        159270.0      2
M-LC       X5K       128219.0      1
M-SC       OX40N     125391.0      4
           XY25      122549.0      1
           XR        120871.0      2
H-MC       X40N      117748.0      3
           XR        114927.0      1
M-LC       XR        102389.0      1
H-LC       XY25       90562.0      3
L-SC       OX40N      88890.0      1
M-SC       X40        82966.0      1
H-LC       X200       68870.0      1
L-MC       XR         58500.0      1
H-SC       MH         54941.0      1
H-MC       OX40N      50732.0      1
H-LC       XY24       46906.0      1
L-LC       XY25       43460.0      1
H-MC       AR         31116.0      1
           MH         30148.0      1
M-LC       XY25       11696.0      1
H-LC       X5K         6501.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
